In [1]:
import os
import pandas as pd
import numpy as np
import random
import gc
from tqdm import tqdm
np.random.seed(2019)
random.seed(2019)



In [31]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 80)

In [5]:
data_path = '/data/workspace/kimi/tencent_ads/2019/dataset'

In [9]:

log_filepath=f'{data_path}/semi/test_tracklog_20190423.last.out'
log_df=pd.read_csv(log_filepath, sep='\t',names=['request_id','request_timestamp','uid','position','aid_info']).sort_values(by='request_timestamp')
print(log_df)



request_id  request_timestamp     uid  position  \
136927      683080         1555948800  202856         2   
10121      1260833         1555948800  185003         1   
50232      4810831         1555948800  203851         2   
248145     4872693         1555948801   58207         2   
139634     2612881         1555948801  294742         2   
202430     1414755         1555948801  225688         2   
123987     4383129         1555948802  339454         2   
201510     3962792         1555948802   68686         2   
138966     3232213         1555948802  318776         2   
201151       99112         1555948802  209683         2   
307214     3671593         1555948803  254689         2   
152621     2770755         1555948803  125987         2   
99778      4509305         1555948804  245102         2   
306191     1054374         1555948804  286889         2   
29343      1754149         1555948804   40613         2   
160594     4256359         1555948805  227177         2   
74045

In [7]:
op_filepath=f'{data_path}/semi/final_map_bid_opt.out'
op_df=pd.read_csv(op_filepath, sep='\t',names=['aid','op_time','op_type','objective','bid_type','bid']).sort_values(by=['aid','op_time'])
print(op_df)


aid         op_time  op_type  objective  bid_type    bid
221156       1  20190411100720        1         10         1  12000
210986       2  20190410130453        2          5         1  13000
222565       3  20190418114808        2          5         1   3400
256173       4  20190422185134        2         10         1   2502
32925        5  20190417133211        1          0         2    100
135956       5  20190418095204        1          0         2    200
102694       5  20190419053334        1          0         2    181
96544        5  20190420053359        1          0         2    188
53631        5  20190421053341        1          0         2    192
186851       5  20190422053322        1          0         2    170
52282        6  20190410000000        1          5         1   8000
308600       8  20190415183758        2          5         2   1500
174436       9  20190415151702        2         13         2   5800
170544      10  20190410194408        2         13         

In [23]:
log_df['aid_list']=  log_df['aid_info'].apply(lambda x:[ i.split(",") for i  in x.split(";")])
del log_df['aid_info']
print(log_df)

request_id  request_timestamp     uid  position                                                                                             aid_list
136927      683080         1555948800  202856         2  [[130356, 69, 69.737, 1394.74, 1463.74, 1], [120824, 54, 54.094, 1081.88, 1135.88, 1], [4400, 53...
10121      1260833         1555948800  185003         1  [[230211, 41, 2.186, 43.72, 133.346, 1], [22896, 37, 2.884, 57.68, 94.68, 1], [77566, 16, 2.294,...
50232      4810831         1555948800  203851         2  [[260233, 4221, 32.229, 644.58, 4865.58, 1], [182839, 2851, 30.99, 619.8, 3470.8, 1], [89490, 20...
248145     4872693         1555948801   58207         2  [[248285, 19, 54.952, 1099.04, 2143.128, 1], [128116, 19, 49.757, 995.14, 1940.523, 1], [264020,...
139634     2612881         1555948801  294742         2  [[105254, 574, 7.267, 145.34, 719.34, 1], [65917, 67, 6.863, 137.26, 597.081, 1], [244806, 96, 4...
202430     1414755         1555948801  225688         2  [[263435,

In [35]:
ids = set()

for item in tqdm(log_df[['request_id','request_timestamp','uid','position','aid_list']].values,total=len(log_df)):
    for it in item[-1]:
        ids.add(int(it[0]))
 

100%|██████████| 312642/312642 [00:15<00:00, 20763.63it/s]


In [49]:
op_df['day']=op_df['op_time']//1000000%100
op_df=op_df.drop_duplicates(['aid','day'],keep='last')
aids=set(op_df[op_df['day']<=21]['aid'])-set(op_df[op_df['day']==22]['aid'])
aids=aids&ids
op_df=op_df[op_df['day']<=21]
op_df=op_df.drop_duplicates('aid',keep='last')
op_df['is']=op_df['aid'].apply(lambda x: x in aids)
op_df=op_df[op_df['is']==True]
op_df=op_df[['aid','objective','bid_type','bid']]
print(op_df)

aid  objective  bid_type    bid
222565       3          5         1   3400
31996       14          0         2     40
267798      30          5         1  20010
1660        44          5         1   8084
248451      48          5         1    722
178819      50          5         2  20010
187425      55          5         2    207
175338      67          5         2    720
29671       81         10         2   3400
71384       89          3         2    400
167319      91          5         2    151
24994      119         10         2   3459
312681     125          5         2    930
60916      126         10         2   2889
314660     129          5         2    600
316190     138          5         2   1200
309515     140         13         1    801
78288      143          5         1    926
177625     144          0         2    201
318419     152         13         2    500
4729       153          5         2    486
132645     163         10         2   6111
80167      170        

In [32]:
def get_imp_count(df):
    imp_count = 0
    for row in df:
        if int(row[-1]) == 1:
            imp_count+=1
    return imp_count

def get_imp_aids(df):
    imp_ids = []
    for row in df:
        if int(row[-1]) == 1:
            imp_ids.append(row[0])
    return imp_ids

log_df['aid_imp_aids']=  log_df['aid_list'].apply(get_imp_aids)
log_df['aid_imp_request_count']=  log_df['aid_list'].apply(lambda x: len(x))
log_df['aid_imp_count']=  log_df['aid_list'].apply(get_imp_count)
print(log_df[log_df['aid_imp_count'] >0 ])

request_id  request_timestamp     uid  position                                                                         aid_list  aid_imp_count                                                                     aid_imp_aids  aid_imp_request_count
136927      683080         1555948800  202856         2  [[130356, 69, 69.737, 1394.74, 1463.74, 1], [120824, 54, 54.094, 1081.88, 11...             11  [130356, 120824, 4400, 126773, 178603, 79255, 167215, 263304, 102659, 263520...                     86
10121      1260833         1555948800  185003         1  [[230211, 41, 2.186, 43.72, 133.346, 1], [22896, 37, 2.884, 57.68, 94.68, 1]...              2                                                                  [230211, 22896]                    165
50232      4810831         1555948800  203851         2  [[260233, 4221, 32.229, 644.58, 4865.58, 1], [182839, 2851, 30.99, 619.8, 34...             62  [260233, 182839, 89490, 190339, 105265, 6949, 171941, 22896, 176070, 259862,...        

In [52]:
imp_dic={}
imp_request={}
for key in aids:
        imp_dic[key]=0
        imp_request[key]=0

log_df = log_df[log_df['aid'] in aids]

for row in tqdm(log_df[['aid_list']].values,total=len(log_df)):
    for ad in row[-1]:
        if int(ad[-1]) == 1:
            imp_dic[int(ad[0])] += 1
            imp_request[int(ad[0])] +=1
        else:
           imp_request[int(ad[0])] +=1

print(len(imp_dic))


KeyError: 'aid'

In [50]:
print(len(imp_dic))
print(len(imp_request))

61042
61042
